## TD single distance

In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import sys
! pip install pmcx
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import pickle
import numpy as np
from pathlib import Path
import os, re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/MCX_data'
sys.path.append(folder_path)
from FD_msNIRS_2layers import extract_freq as ef

Mounted at /content/drive


### GT

In [ ]:
train_sorted_ids = [i+1 for i in range(10000)]
GT_folder_train = '/content/drive/MyDrive/MCX_data/stage2_csv/'
csv_files = glob.glob(os.path.join(GT_folder_train, '*.csv'))
GT_all_train = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

# Ensure ID column is integer
GT_all_train['ID'] = GT_all_train['ID'].astype(int)

# Filter and sort GT_all to match the sorted_ids order
GT_filtered = GT_all_train[GT_all_train['ID'].isin(train_sorted_ids)].copy()
GT_filtered = GT_filtered.set_index('ID').loc[train_sorted_ids].reset_index()

# Extract HHB2 as Y
Y_train = GT_filtered['HHB2'].values  # numpy array (n_samples,)
Y_train = Y_train.reshape(-1, 1)
print(Y_train.shape)

(10000, 1)


In [ ]:
test_sorted_ids = [i+1 for i in range(1000)]
GT_folder_test = '/content/drive/MyDrive/MCX_data/test_csv/'
csv_files = glob.glob(os.path.join(GT_folder_test, '*.csv'))
GT_all_test = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

GT_all_test['ID'] = GT_all_test['ID'].astype(int)

GT_filtered = GT_all_test[GT_all_test['ID'].isin(test_sorted_ids)].copy()
GT_filtered = GT_filtered.set_index('ID').loc[test_sorted_ids].reset_index()

Y_test = GT_filtered['HHB2'].values  # numpy array (n_samples,)
Y_test = Y_test.reshape(-1, 1)
print(Y_test.shape)

(1000, 1)


#### scale X and Y

In [ ]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler_y  = StandardScaler()
Y_train = scaler_y.fit_transform(Y_train.reshape(-1, 1)).ravel()
Y_test  = scaler_y.transform(Y_test.reshape(-1, 1)).ravel()

In [ ]:
pth = '/content/drive/MyDrive/MCX_data/weights_td/'
joblib.dump(scaler_y, pth+"TD_thickness_CNN_scaler_y.pkl")
SAVE_DIR = Path("/content/drive/MyDrive/MCX_data/splits")
data    = np.load(SAVE_DIR / "train_test_split.npz", allow_pickle=False)
X_train = data["X_train"]
X_test  = data["X_test"]

In [ ]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(10000, 64, 1000) (1000, 64, 1000) (10000,) (1000,)


### Create Seperations

In [ ]:
import numpy as np

def split_by_distance(
        X_train: np.ndarray,   # shape (N_train, 64, 1000)
        X_test : np.ndarray    # shape (N_test , 64, 1000)
    ) -> dict[int, list[np.ndarray]]:
    """
    Break the 64-channel TPSF cube into 8×8-channel blocks (one per S-D distance).

    Returns
    -------
    dist_dict : dict
        Keys 1 … 8  →  value is [ X_train_slice , X_test_slice ]
        Each slice has shape (N, 8, 1000).
    """
    if X_train.shape[1] != 64 or X_test.shape[1] != 64:
        raise ValueError("Expect second dimension = 64 (8 distances × 8 wavelengths).")

    dist_dict: dict[int, list[np.ndarray]] = {}
    for d in range(1, 9):                       # distances 1 … 8
        start = (d - 1) * 8                     # rows 0-7, 8-15, …
        end   = d * 8
        dist_dict[d] = [
            X_train[:, start:end, :],           # (N_train, 8, 1000)
            X_test[:,  start:end, :]
        ]
    return dist_dict


# ── usage ──────────────────────────────────────────────────────────
# X_train.shape == (10000, 64, 1000)
# X_test .shape == (1000 , 64, 1000)

distance_data = split_by_distance(X_train, X_test)

# e.g. distance_data[3][0]  → training samples for distance-3 (shape 10000×8×1000)
#      distance_data[3][1]  → test     samples for distance-3 (shape  1000×8×1000)


In [ ]:
distance_data.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8])

### ANN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, Activation, Add,
    MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout
)
from pathlib import Path
import joblib, tensorflow as tf


def residual_block(x, filters, kernel_size=(5, 5), downsample=False, dropout_rate=0.0):
    shortcut = x

    # First conv
    x = Conv2D(filters, kernel_size, padding='same', strides=(2, 2) if downsample else (1, 1))(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_rate)(x)

    # Second conv
    x = Conv2D(filters, kernel_size, padding='same')(x)

    # Shortcut path
    if downsample or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), strides=(2, 2) if downsample else (1, 1), padding='same')(shortcut)

    # Merge
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

# Input
input_tensor = Input(shape=(8, 1000, 1))

# Initial Conv Layer
x = Conv2D(32, (3, 3), padding='same', activation='relu')(input_tensor)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Residual Blocks
x = residual_block(x, 128, downsample=True, dropout_rate=0.0)
x = residual_block(x, 128)

x = residual_block(x, 128, downsample=True, dropout_rate=0.0)
x = residual_block(x, 128)

x = residual_block(x, 256, downsample=True, dropout_rate=0.0)
x = residual_block(x, 256)

# Global Average Pooling
x = GlobalAveragePooling2D()(x)

# Dense Head
x = Dense(128, activation='relu')(x)
x = Dropout(0.0)(x)
x = Dense(64, activation='relu')(x)
x = Dense(1)(x)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
Path = '/content/drive/MyDrive/MCX_data/'
for i in range (1, 9):
  # Build model
     resnet_model = Model(inputs=input_tensor, outputs=x)

# 1) Create a checkpoint callback that saves the best model:
     checkpoint_cb = ModelCheckpoint(
        "TD_hhb2_CNN.keras",        # filepath to save to
        monitor="val_loss",     # quantity to monitor
        mode="min",             # we want to minimize val_loss
        save_best_only=True,    # only save when val_loss decreases
        save_weights_only=False # set True if you only want weights
     )
     optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
     resnet_model.compile(optimizer=optimizer, loss="mse")
     history = resnet_model.fit( distance_data[i][0], Y_train, epochs=200, batch_size=32, validation_split=0.1, verbose=1, callbacks=[checkpoint_cb])
     resnet_model = tf.keras.models.load_model("TD_hhb2_CNN.keras")
     y_pred = resnet_model.predict(distance_data[i][1])
     y_pred = np.clip(y_pred, scaler_y.transform(np.array([[15]]))[0][0], scaler_y.transform(np.array([[35]]))[0][0])
     ###
     y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).flatten()
     y_test = scaler_y.inverse_transform(Y_test.reshape(-1, 1)).flatten()
     mse = mean_squared_error(y_test, y_pred)
     print(f"MSE for distance {i}: {mse}")
     print(f"RMSE for distance {i}: {np.sqrt(mse)}")
     df = pd.DataFrame({
        "y_test" : y_test,
        "y_pred" : y_pred
    })
     df.to_csv(Path + f"distance_{i}_hhb2_TD.csv", index=False)

Epoch 1/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - loss: 0.9933 - val_loss: 0.9269
Epoch 2/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.7315 - val_loss: 0.6083
Epoch 3/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.5172 - val_loss: 0.4807
Epoch 4/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.4759 - val_loss: 0.3894
Epoch 5/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.4188 - val_loss: 0.4642
Epoch 6/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.3761 - val_loss: 0.3250
Epoch 7/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.3344 - val_loss: 0.3493
Epoch 8/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.3009 - val_loss: 0.3333
Epoch 9/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.2791 - val_loss: 0.3346
Epoch 10/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.2694 - val_loss: 0.3426
Epoch 11/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.2449 - val_loss: 0.2764
Epoch 12/200
282/282 ━━━━━━━━━